In [50]:
import numpy as np
import tensorflow

data = np.load("./train.npz")
test = np.load("./test.npz")
print(data.files)
print(test.files)

x_train = data['x']
y_train = data['y']

x_train = x_train.reshape(-1, 500, 500, 1) / 255.0
test_data = test['x'].reshape(-1, 500, 500, 1)/ 255.0

print(type(x_train))
print(y_train.shape)

['x', 'y']
['x']
<class 'numpy.ndarray'>
(150,)


In [51]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

print(y_train[:10])
le = LabelEncoder()
y_train = le.fit_transform(y_train)



print(y_train[:10])

['normal' 'normal' 'normal' 'pneumonia' 'normal' 'normal' 'pneumonia'
 'pneumonia' 'normal' 'normal']
[0 0 0 1 0 0 1 1 0 0]


In [52]:
from sklearn.model_selection import train_test_split

# 원본에서 직접 분리 (shuffle 자동)
x_train, x_val, y_train, y_val = train_test_split(
    x_train, y_train,
    test_size=0.2,   # 20%를 validation
    random_state=42,
    shuffle=True,
    stratify=y_train,
)

In [53]:
from tensorflow import keras
from keras import Sequential, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

In [54]:
from sklearn.utils.class_weight import compute_class_weight

# 자동으로 가중치 계산
class_weights = compute_class_weight(
    'balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weight_dict = {0: class_weights[0], 1: class_weights[1]}

print("가중치:", class_weight_dict)
# 예: {0: 0.56, 1: 4.5} ← class 1이 적으니까 4.5배 페널티

가중치: {0: 1.0, 1: 1.0}


In [55]:
from tensorflow.keras import backend as K
K.clear_session()

# 1. 모델 구조 단순화 + Regularization 강화
model = Sequential([
    Input(shape=(500,500,1)),

    Conv2D(128, 3, padding='same', activation='relu'),
    # BatchNormalization(),
    MaxPooling2D(),

    Conv2D(64, 3, padding='same', activation='relu'),
    # BatchNormalization(),
    MaxPooling2D(),


    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(
    optimizer=Adam(learning_rate=0.005),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)


train_datagen = ImageDataGenerator(
#    height_shift_range=0.05,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator()

train_generator = train_datagen.flow(
    x_train, y_train,
    batch_size=32,
    shuffle=True
)

val_generator = val_datagen.flow(
    x_val, y_val,
    batch_size=32,
    shuffle=False
)

# 4. Early Stopping 더 엄격하게
checkpoint_cb = ModelCheckpoint(
    "best-cnn-model.keras",
    monitor='val_loss',
    save_best_only=True
)

early_stopping_cb = EarlyStopping(
    patience=5,
    restore_best_weights=True,
    monitor='val_loss'
)

# 6. 학습
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=val_generator,
    callbacks=[checkpoint_cb, early_stopping_cb],
    verbose=1
)

Epoch 1/20


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


4/4 ━━━━━━━━━━━━━━━━━━━━ 47s 13s/step - accuracy: 0.4961 - loss: 179.5947 - val_accuracy: 0.5000 - val_loss: 0.6979
Epoch 2/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 54s 14s/step - accuracy: 0.6492 - loss: 0.5910 - val_accuracy: 0.9000 - val_loss: 0.2696
Epoch 3/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 49s 13s/step - accuracy: 0.7556 - loss: 1.1064 - val_accuracy: 0.7667 - val_loss: 0.4444
Epoch 4/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 48s 12s/step - accuracy: 0.8266 - loss: 0.3880 - val_accuracy: 0.6667 - val_loss: 0.5129
Epoch 5/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 40s 10s/step - accuracy: 0.8388 - loss: 0.3540 - val_accuracy: 0.8333 - val_loss: 0.5624
Epoch 6/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 37s 10s/step - accuracy: 0.8058 - loss: 0.3959 - val_accuracy: 0.7333 - val_loss: 0.5167
Epoch 7/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 37s 9s/step - accuracy: 0.8176 - loss: 0.4420 - val_accuracy: 0.7333 - val_loss: 0.4456


In [56]:
print("Train:", np.unique(y_train, return_counts=True))
print("Val:", np.unique(y_val, return_counts=True))

# 2. 예측 결과 확인
predictions = model.predict(x_val)
pred_classes = (predictions > 0.5).astype(int)
print("예측 분포:", np.unique(pred_classes, return_counts=True))
print("실제 분포:", np.unique(y_val, return_counts=True))

Train: (array([0, 1]), array([60, 60]))
Val: (array([0, 1]), array([15, 15]))
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
예측 분포: (array([0, 1]), array([18, 12]))
실제 분포: (array([0, 1]), array([15, 15]))


In [57]:
y_pred = model.predict(test_data)
print(y_pred)

2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 384ms/step
[[7.7682179e-01]
 [4.8555392e-03]
 [7.3784631e-01]
 [6.7226094e-01]
 [7.8557742e-01]
 [1.2706806e-03]
 [1.7155677e-01]
 [5.6442547e-01]
 [6.7041022e-01]
 [2.2714107e-01]
 [3.7908638e-01]
 [1.0552392e-03]
 [4.3356991e-03]
 [8.5309980e-04]
 [7.6257548e-04]
 [8.3607626e-01]
 [9.6811050e-01]
 [5.7262082e-02]
 [4.0304977e-01]
 [9.1114974e-01]
 [3.2422706e-01]
 [1.9978000e-02]
 [5.6893331e-01]
 [2.4504866e-03]
 [9.1598288e-04]
 [7.0797659e-02]
 [2.3675000e-03]
 [6.6505730e-01]
 [8.0603367e-01]
 [9.4905543e-01]
 [8.5391051e-01]
 [7.6472777e-01]
 [1.1628062e-01]
 [7.8608532e-04]
 [4.1746390e-03]
 [9.3121767e-01]
 [8.5536945e-01]
 [1.3495184e-03]]


In [58]:
import numpy as np

y_pred_classes = (y_pred > 0.5).astype(int)
print(y_pred_classes)
print(len(y_pred_classes))

[[1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]]
38


In [59]:
y_pred_classes = y_pred_classes.flatten().tolist()
y_list = ['normal', 'pneumonia']
y = [y_list[i] for i in y_pred_classes]
print(y)

['pneumonia', 'normal', 'pneumonia', 'pneumonia', 'pneumonia', 'normal', 'normal', 'pneumonia', 'pneumonia', 'normal', 'normal', 'normal', 'normal', 'normal', 'normal', 'pneumonia', 'pneumonia', 'normal', 'normal', 'pneumonia', 'normal', 'normal', 'pneumonia', 'normal', 'normal', 'normal', 'normal', 'pneumonia', 'pneumonia', 'pneumonia', 'pneumonia', 'pneumonia', 'normal', 'normal', 'normal', 'pneumonia', 'pneumonia', 'normal']


In [60]:
import pandas as pd
import numpy as np

df = pd.read_csv("submission.csv")

df.dropna(axis=1, inplace=True)

# 아래 "np.arange(0, df.shape[0], 1)" 부분을, 솜솜이가 만든 모델이 예측한 값으로 대체!
#y_pred = np.arange(0, df.shape[0], 1) 

df["result"] = y
df.to_csv("new_submission.csv", index=False)